In [1]:
from pdf2image import convert_from_path
import pandas as pd
import fitz, re
from PIL import Image
import easyocr
from datetime import datetime

In [7]:
def scrapPdf(nombre):
  
    # leer pdf
    doc = fitz.open(nombre)
    page = doc.loadPage(0) 
    text = page.getText('text')
    # extración  información página 0
    a1 = 'La desagregación'
    b1 = 'INFOGRAFÍA'
    ca1 = text.index(a1) + len(a1)
    cb1 = text.index(b1)
    infoParcial = text[cb1:ca1].split('\n')
    auxInfo =infoParcial[1].split('-')
    info = [infoParcial[0].split(' ')[1][2:],auxInfo[1][6:17].strip(),auxInfo[0][6:17].strip()]
    numInfo = info[0]
    fecha = info[1]
    namef = datetime.strptime(fecha, '%d/%m/%Y').date().strftime('%A')

    cadena_A = 'Casos confirmados por grupo etario'
    cadena_B = 'SITUACIÓN NACIONAL POR  COVID-19' 
    c_A = text.index(cadena_A)
    c_B = text.rindex(cadena_B)
    datos_provincias = text[c_A:c_B]
    datos_provincias= datos_provincias[len(cadena_A):]
    if 'Azuay' in datos_provincias:
        datos_provincias = datos_provincias[datos_provincias.index('Azuay'):datos_provincias.index('Total')]

        f = lambda A, n=4: [A[i:i+n] for i in range(0, len(A), n)]
        salidatexto = datos_provincias.split('\n')
        salida2 = f(salidatexto)
        salida2 = salida2[:-1]

        p = list(map(lambda x: x[0],salida2 ) )
        c = list(map(lambda x: x[1].replace('.','') ,salida2 ) )
        f = list(map(lambda x: x[2].replace('.',''),salida2 ) )
        fp = list(map(lambda x: x[3].replace('.',''),salida2 ))

        out2Dict = {'Nro.Infografía':numInfo,'fecha':fecha,'Provincia':p,'Confirmados':c,'Fallecidos':f,'FallecidosProbables':fp,'NombreDia':namef}
        oi = pd.DataFrame(out2Dict)
        return  oi
    else:
        poppler_path='C:/Program Files/poppler-0.68.0/bin'
        pages = convert_from_path(nombre, dpi =500,poppler_path=poppler_path,first_page =0 ,last_page=1)
        pages[0].save('out.jpg', 'JPEG')
        img = Image.open('out.jpg')
        outcut = img.crop((1585,1220,3290,2920))
        outcut.save('outcut.jpg', 'JPEG')
        reader = easyocr.Reader(['es'])
        dd= reader.readtext('outcut.jpg')
        
        texto_aux = ''
        for r in dd:
            texto_aux += r[1] + ' '
        texto_aux =texto_aux.replace('.','')
#         end_line = texto_aux[texto_aux.index('Total'):].strip().rsplit(' ',3)# el-> end line
        list_pro = re.findall('[A-Z]\D+ \d+ \d+ \d+',texto_aux)
        list_list_pro = list(map(lambda x: x.strip().rsplit(' ',3),list_pro))
        for y in list_list_pro:
            if len(y)==2 :
                y.append('0')
            if len(y)==3 :
                y.append('0')
#         list_list_pro.append(end_line)
        p2 = list(map(lambda z: z[0],list_list_pro ) )
        c2 = list(map(lambda z: z[1] ,list_list_pro ) )
        f2 = list(map(lambda z: z[2].replace(',','') , list_list_pro ) )
        fp2 = list(map(lambda z: z[3].replace(',',''), list_list_pro ))
        
        out2Dict2 = {'Nro.Infografía':numInfo,'fecha':fecha,'Provincia':p2,'Confirmados':c2,'Fallecidos':f2,'FallecidosProbables':fp2,'NombreDia':namef}
        oe = pd.DataFrame(out2Dict2)
        return  oe
        
    

In [8]:
#abrir csv con nombre de infogrfias
df = pd.read_csv('C:/Users/Hugo Alvarez T/Documents/AnalisisDatos/Extraccion/dataNamePdfs.csv')

# nombres = list(df.query('numInfo >= 332 and numInfo <= 350')['nombreArchivo'])
nombres = list(df.query('numInfo >= 332 and numInfo <= 350')['nombreArchivo'])
n =[ x+'.pdf' for x in nombres]#pn ->representa path nombres
print(len(n),len(nombres))

19 19


In [9]:
listdf = []
aux = pd.DataFrame()
count  = 0
for i, e in enumerate(n):
    try:
        listdf.append(scrapPdf(n[i]))
    except:
        count = count + 1
        print("\nNo se pudo abrir el archivo: {} ".format(e))
        
print('Nro. de archivos no leidos: ',count)
massExtraction = pd.concat( listdf, ignore_index = False)

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


Nro. de archivos no leidos:  0


In [10]:
massExtraction

,Nro.Infografía,fecha,Provincia,Confirmados,Fallecidos,FallecidosProbables,NombreDia
0,350,11/02/2021,Azuay,16521,293,25,Thursday
1,350,11/02/2021,Bolívar,3252,76,15,Thursday
2,350,11/02/2021,Cañar,3699,109,6,Thursday
3,350,11/02/2021,Carchi,5303,173,3,Thursday
4,350,11/02/2021,Chimborazo,4482,387,126,Thursday
...,...,...,...,...,...,...,...
18,332,24/01/2021,Santa Elena,2979,406,280,Sunday
19,332,24/01/2021,Sto Domingo Tsác,6609,401,118,Sunday
20,332,24/01/2021,Sucumbíos,3183,107,3,Sunday
21,332,24/01/2021,Tungurahua,8775,380,314,Sunday


In [12]:
file= open('C:/Users/Hugo Alvarez T/Documents/AnalisisDatos/Extraccion/dataFinalTest.csv')
file.close
massExtraction.to_csv('C:/Users/Hugo Alvarez T/Documents/AnalisisDatos/Extraccion/dataFinalTest.csv',encoding ='utf-8-sig',index=False)